# Código 08: Matrizes de translação, rotação e escala

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [41]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np
import random

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
WIDTH : int = 800
HEIGHT : int = 800
window = glfw.create_window(WIDTH, HEIGHT, "Transformação Geométrica", None, None)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

Criaremos um qualificador `uniform` para armazenar a matriz de transformação que será aplicado a todos os vértices. Também fazemos a matriz multiplicar a posição (vide teoria de matriz de transformação)

In [42]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Solicitando espaço, compilando e linkando

In [43]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

In [44]:
#Criando espaço
vertices = np.zeros(3, [("position", np.float32, 2)])

#Preenchendo as coordenadas
vertices['position'] = [
                            ( 0.00, +0.05), 
                            (-0.05, -0.05), 
                            (+0.05, -0.05)
]

### Manipulação dos espaços de dados

In [45]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_position = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_position)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_position, 2, GL_FLOAT, False, stride, offset)

### Capturando localização do qualificador `uniform` responsável pela cor e pela matriz de transformação

In [46]:
loc_color = glGetUniformLocation(program, "color")
loc_mat = glGetUniformLocation(program, "mat_transformation")

### Eventos de teclado e mouse

In [47]:
# Coordenadas do elemento
t_x = t_y = 0

# Coordenadas do cursor
c_x = c_y = 0

# Escala do elemento
scale = 1.0

# Rotação do elemento
angle = 0.0

# Cores
R = G = B = random.uniform(0.5,1.0)

def key_event(window,key,scancode,action,mods):
    # Global: variáveis não estão sendo criadas como locais dentro da função
        # mas sim referenciando as variáveis globais definidas fora da função.
    global t_x, t_y, scale, angle, R, G, B
    
    linear_speed = 0.05
    if key == 265: t_y += linear_speed #Seta p/cima
    if key == 264: t_y -= linear_speed #Seta p/baixo
    if key == 263: t_x -= linear_speed #Seta p/esquerda
    if key == 262: t_x += linear_speed #Seta p/direita
    
    scale_speed = 0.2
    if key == 65: scale += scale_speed #Tecla A
    if key == 83: scale -= scale_speed #Tecla S
    
    angle_speed = 0.139 #radianos
    if key == 68: angle += angle_speed #Tecla D
    if key == 70: angle -= angle_speed #Tecla F

    color_speed = 0.1
    if mods == 1:
        if key == 82: R += color_speed #Tecla R
        if key == 71: G += color_speed #Tecla G
        if key == 66: B += color_speed #Tecla B
    else:
        if key == 82: R -= color_speed #Tecla R
        if key == 71: G -= color_speed #Tecla G
        if key == 66: B -= color_speed #Tecla B

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    
glfw.set_key_callback(window, key_event)

def mouse_event(window,button,action,mods):
    global t_x, t_y, c_x, c_y
    if button == 1: 
        t_x = c_x
        t_y = c_y
    # print('[mouse event] button=',button)
    # print('[mouse event] action=',action)
    # print('[mouse event] mods=',mods)
    # print('-------')

glfw.set_mouse_button_callback(window, mouse_event)

def cursor_event(window, xpos, ypos):
    global c_x, c_y
    c_x = xpos*(2/WIDTH) - 1
    c_y = 1 - ypos*(2/HEIGHT)
    #print(f"Posição do cursor: ({c_x}, {c_y})")

glfw.set_cursor_pos_callback(window, cursor_event)


### Exibindo na tela

In [48]:
glfw.show_window(window)

## Loop principal

* `glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_transformation)`: função usada para alterar o conteúdo do qualificador `uniform` com localização `loc_mat`.

In [49]:
def calc_mat_transform(a,b,c):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = c.reshape(4,4)
    return np.dot(np.dot(m_a, m_b), m_c)

def calc_angle_direction(c_x, c_y, t_x, t_y):
    # Calcular a direção do objeto para o cursor
    direction = np.array([c_x, c_y]) - np.array([t_x, t_y])
    return np.arctan2(direction[1], direction[0])

def calc_limits_coord(t_x, t_y):
    BORDA_SUP : int = 1
    BORDA_INF : int = -1
    
    if t_y >= BORDA_SUP:
        t_y = BORDA_SUP
    elif t_y <= BORDA_INF:
        t_y = BORDA_INF

    if t_x >= BORDA_SUP:
        t_x = BORDA_SUP
    elif t_x <= BORDA_INF:
        t_x = BORDA_INF
    
    return t_x, t_y

In [50]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    # glPolygonMode(GL_FRONT,GL_LINE)
    # glPolygonMode(GL_BACK,GL_LINE)
    # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Recalcula t_x e t_y caso ultrapassem a borda
    t_x, t_y = calc_limits_coord(t_x,t_y)

    # > Matriz de rotacao
    angle = calc_angle_direction(c_x, c_y, t_x, t_y)
    cos = np.cos(angle)
    sin = np.sin(angle)
    mat_rotation = np.array([  cos, -sin, 0.0, 0.0, 
                               sin,  cos, 0.0, 0.0, 
                               0.0,  0.0, 1.0, 0.0, 
                               0.0,  0.0, 0.0, 1.0], np.float32)
    
    # > Matriz de escala
    # Declara variável scale_x e scale_y para serem aplicados a cada eixo
    # Como queremos uma variação igual de escala no dos eixos, eles são iguais
    scale_x = scale_y = scale
    mat_scale =    np.array([  scale_x,     0.0, 0.0, 0.0, 
                                   0.0, scale_y, 0.0, 0.0, 
                                   0.0,     0.0, 1.0, 0.0, 
                                   0.0,     0.0, 0.0, 1.0], np.float32)
    
    # > Matriz de translação
    mat_translation = np.array([  1.0, 0.0, 0.0, t_x, 
                                  0.0, 1.0, 0.0, t_y, 
                                  0.0, 0.0, 1.0, 0.0, 
                                  0.0, 0.0, 0.0, 1.0], np.float32)
    
    #> Cálculo da matriz de transformação
    #É necessário colocar a matriz de rotação mais afastado do ponto na hora do cálculo
    mat_transformation = calc_mat_transform(mat_translation, mat_rotation, mat_scale)
    
    #Aplica transformações, cores e rasteriza
    glUniformMatrix4fv(loc_mat, 1, GL_TRUE, mat_transformation)
    glUniform4f(loc_color, R, G, B, 1.0)
    glDrawArrays(GL_TRIANGLES, 0, 3)

    glfw.swap_buffers(window)

glfw.terminate()

#Adicionar mais de um elemnto ("tiro" circular)
#Melhorar rotação com mouse